In [2]:
import keras.utils
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
import pandas as pd
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.model_selection import train_test_split
from keras.utils import pad_sequences
import numpy as np
from sklearn.metrics import classification_report
from collections import Counter
import joblib
import tensorflow as tf

In [15]:
data = pd.read_csv('../ds/sentiment_data/combined_data.csv')

# Split the data into features (X) and labels (y)
X = data['Nhận xét đánh giá'].values
y = data['Cảm xúc'].values
label_encoder = LabelEncoder()

# Encode the labels
y_encoded = label_encoder.fit_transform(y)
y_new_encoded_series = pd.Series(y_encoded)
unique_values = y_new_encoded_series.unique()
print(unique_values)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=132)



[0 2 1 3 4 5]


In [4]:
borderline_smote = BorderlineSMOTE(sampling_strategy='auto', random_state=132)

max_words = 5000
max_len = 1000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

X_train_seq = pad_sequences(X_train_seq, maxlen=max_len)
X_val_seq = pad_sequences(X_val_seq, maxlen=max_len)
X_train_resampled, y_train_resampled = borderline_smote.fit_resample(X_train_seq, y_train)

In [5]:
class_distribution_before_SMOTE = Counter(y_train)

class_distribution_after_SMOTE = Counter(y_train_resampled)

print("Class Distribution Before SMOTE:")
for label, count in class_distribution_before_SMOTE.items():
    print(f"Class {label}: {count} samples")

# Print class distribution after SMOTE
print("\nClass Distribution After SMOTE:")
for label, count in class_distribution_after_SMOTE.items():
    print(f"Class {label}: {count} samples")

Class Distribution Before SMOTE:
Class 0: 3322 samples
Class 3: 2976 samples
Class 5: 918 samples
Class 1: 3594 samples
Class 4: 1269 samples
Class 2: 3209 samples

Class Distribution After SMOTE:
Class 0: 3594 samples
Class 3: 3594 samples
Class 5: 3594 samples
Class 1: 3594 samples
Class 4: 3594 samples
Class 2: 3594 samples


In [6]:

# Build and compile your CNN model
embedding_dim = 120
num_filters = 128
kernel_size = 5
num_classes = len(np.unique(y_train_resampled))
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(Conv1D(num_filters, kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(num_classes, activation='softmax'))

optimizer = keras.optimizers.Adam(learning_rate=0.0007)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [7]:
batch_size = 128
epochs = 10

model.fit(X_train_seq, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val_seq, y_val))

Epoch 1/10
120/120 [==============================] - 97s 784ms/step - loss: 1.3947 - accuracy: 0.4971 - val_loss: 1.0367 - val_accuracy: 0.6123
Epoch 2/10
120/120 [==============================] - 123s 1s/step - loss: 0.8766 - accuracy: 0.6854 - val_loss: 0.8081 - val_accuracy: 0.7070
Epoch 3/10
120/120 [==============================] - 119s 990ms/step - loss: 0.7069 - accuracy: 0.7474 - val_loss: 0.7372 - val_accuracy: 0.7361
Epoch 4/10
120/120 [==============================] - 114s 954ms/step - loss: 0.6292 - accuracy: 0.7751 - val_loss: 0.7102 - val_accuracy: 0.7403
Epoch 5/10
120/120 [==============================] - 114s 950ms/step - loss: 0.5695 - accuracy: 0.8002 - val_loss: 0.6976 - val_accuracy: 0.7541
Epoch 6/10
120/120 [==============================] - 117s 972ms/step - loss: 0.5226 - accuracy: 0.8172 - val_loss: 0.6964 - val_accuracy: 0.7518
Epoch 7/10
120/120 [==============================] - 114s 954ms/step - loss: 0.4847 - accuracy: 0.8301 - val_loss: 0.6908 - val

In [12]:
y_pred = model.predict(X_val_seq)
# Convert numerical labels to original labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_labels)

# Convert true labels to original labels
y_val_labels = label_encoder.inverse_transform(y_val)

# Generate classification report
report = classification_report(y_val_labels, y_pred_labels)
print(report)

120/120 [==============================] - 8s 69ms/step
              precision    recall  f1-score   support

           1       0.68      0.68      0.68       787
           2       0.71      0.74      0.72       923
           3       0.75      0.69      0.72       828
           4       0.84      0.89      0.87       723
           5       0.79      0.75      0.77       334
           6       0.93      0.96      0.94       228

    accuracy                           0.76      3823
   macro avg       0.78      0.78      0.78      3823
weighted avg       0.76      0.76      0.76      3823



In [13]:
# model.save('../saved_model/sentiment_model')
model.save('../saved_model/sentiment_model.keras')

In [10]:
custom_text = "thay giao day rat tuyet"

# Tokenize and pad the custom text
custom_text_seq = tokenizer.texts_to_sequences([custom_text])
custom_text_seq = keras.utils.pad_sequences(custom_text_seq, maxlen=max_len)

# Make predictions on the custom text
custom_text_pred = model.predict(custom_text_seq)

# Convert numerical labels to original labels
custom_text_pred_label = np.argmax(custom_text_pred, axis=1)
custom_text_pred_label = label_encoder.inverse_transform(custom_text_pred_label)

# Print the predicted label
print(f"Predicted Sentiment: {custom_text_pred_label[0]}")

1/1 [==============================] - 0s 67ms/step
Predicted Sentiment: 2


In [14]:
new_model = tf.keras.models.load_model('../saved_model/sentiment_model.keras')

# Check its architecture
new_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 120)         600000    
                                                                 
 conv1d (Conv1D)             (None, 996, 128)          76928     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 6)                 774       
                                                                 
Total params: 677,702
Trainable params: 677,702
Non-trainable params: 0
_________________________________________________________________
